In [6]:
import pandas as pd
import os
import json
import string as str

In [4]:
#mount Google driver if needed
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
# datasets from 2020 <https://webrobots.io/kickstarter-datasets/>
FILE_PATH_DEC = 'https://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2020-12-17T03_20_12_051Z.zip'


## functions

In [11]:
def unzipFile(downloadLink, fileName):
  
  newDir = '/content/{}'.format(fileName) 
  %cd
  %mkdir {newDir}
  %cd {newDir}
  print('\n')
  print('created new directory: {}'.format(newDir))
  print('\n')
  !wget -O {fileName} {downloadLink}
  !unzip -q {fileName}
  fileCount = !find . -name 'Kickstarter0*.csv' | wc -l
  fileCount = int(fileCount[0])
  print('\n')
  print('number of files unziped from {}: {}'.format(fileName, fileCount))

  return fileCount


In [12]:
# assume pwd is fileName directory
def appendMonthlyData(fileCount):

  data = pd.read_csv('Kickstarter.csv')
  for x in range(fileCount):
    df = pd.read_csv('Kickstarter0{0:0=2d}.csv'.format(x+1))
    data = data.append(df)
    #print('added Kickstarter0{0:0=2d}.csv; new df shape:'.format(x+1), data.shape)
  print('final dataframe size: {}'.format(data.shape))
  
  return data


In [13]:
def wrangle(df):
  
  #drop duplicated rows based on 'id' column; only keep first occurence
  df.drop_duplicates(subset=['id'],inplace=True)

  #fix index column
  df.drop('Unnamed: 0',axis=1,inplace=True)
  df.set_index('id',inplace=True)

  #remove columns with too many null values
  df.drop(columns=['friends', 'is_starred', 'permissions'], inplace=True)
  
  #remove columns with constant values
  df.drop(columns=['disable_communication','is_backing'], inplace=True)

  #remove columns with redundant information or information that cannot be incoporated into model
  df.drop(columns=[
                        'created_at',
                        'converted_pledged_amount',
                        'country_displayable_name',
                        'creator',
                        'currency',
                        'currency_symbol',
                        'currency_trailing_code',
                        'current_currency',
                        'fx_rate',
                        'is_starrable',
                        'location',
                        'name',
                        'photo',
                        'pledged',
                        'profile',
                        'source_url',
                        'state_changed_at',
                        'urls',
                        'usd_pledged',
                        'usd_type'
                        ], inplace=True)
  
  #remove columns that may have leaky data
  df.drop(columns=['backers_count','spotlight'], inplace=True)
  
  #all date columns from original dataset are in unix format
  #this function converts it to DateTime format
  def unixToDateTime(colName):
    pdDate = pd.to_datetime(df[colName], origin='unix', unit='s').dt.date
    dateTime = pd.to_datetime(pdDate,format='%Y-%m-%d')
    return dateTime
  
  df['deadline'] = unixToDateTime('deadline')
  df['launched_at'] = unixToDateTime('launched_at')

  #engineer new column 'campaignPeriod' and drop used columns
  df['campaignPeriod'] = df['deadline']-df['launched_at']
  #change 'campaignPeriod' to int type
  df['campaignPeriod'] = df['campaignPeriod'].dt.days
  df.drop(columns=['deadline','launched_at'],inplace=True)

  #engineer new column 'goalUsd' and drop used columns
  df['goalUsd'] = df['goal'] * df ['static_usd_rate']
  df.drop(columns=['goal','static_usd_rate'],inplace=True)

  #engineer new column 'sub_category_name' and 'main_category_name'
  #drop column 'category'
  def CustomParserSub(data):
    j1 = json.loads(data)
    return j1.get('name')
  df['sub_category_name'] = df['category'].apply(CustomParserSub)
  
  def CustomParserMain(data):
    j1 = json.loads(data)
    return j1.get('parent_name')
  df['main_category_name'] = df['category'].apply(CustomParserMain)
  
  df.drop(columns='category',inplace=True)

  #drop rows where project 'state' is 'live' or 'canceled'
  df = df[(df['state'] == 'successful') | (df['state'] == 'failed')]

  #encode target variable
  df['state'] = df['state'].apply(lambda x: 1 if x=='successful' else 0)


  return df

## load file & wrangle

In [ ]:
#download and unzip file
fileCount = unzipFile(FILE_PATH_DEC, '2020-12-17')

In [ ]:
#append all unzipped files into a single df
df = appendMonthlyData(fileCount)

final dataframe size: (217253, 38)


In [ ]:
#save df as csv file to Google drive
%cd /content/drive/My Drive/kickStarter
df.to_csv('rawdf.csv')

Mounted at /content/drive/
/content/drive/My Drive/kickStarter


In [14]:
#to RELOAD df...
%cd /content/drive/My Drive/kickStarter
df = pd.read_csv('rawdf.csv')
df.shape

/content/drive/My Drive/kickStarter


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15,19,21,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(217253, 39)

In [16]:
df = wrangle(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180679 entries, 1124922936 to 1126079486
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   blurb               180677 non-null  object 
 1   country             180679 non-null  object 
 2   slug                180679 non-null  object 
 3   staff_pick          180679 non-null  bool   
 4   state               180679 non-null  int64  
 5   campaignPeriod      180679 non-null  int64  
 6   goalUsd             180679 non-null  float64
 7   sub_category_name   180679 non-null  object 
 8   main_category_name  174232 non-null  object 
dtypes: bool(1), float64(1), int64(2), object(5)
memory usage: 12.6+ MB


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [17]:
df.head()

,blurb,country,slug,staff_pick,state,campaignPeriod,goalUsd,sub_category_name,main_category_name
id,,,,,,,,,
1124922936,Bringing deliciously fresh snacking to every g...,US,palmetto-pork-skins,False,0,30,25000.000,Food Trucks,Food
1479488174,Warm hands and full access to your camera dial...,NO,photography-gloves-extend-your-session-in-style,False,1,32,13905.834,Camera Equipment,Technology
2077771673,Enamel pins dedicated to my Grandpa and his di...,US,eat-shit-a-memorial-pin-campaign,False,1,31,250.000,Accessories,Fashion
2008689997,"To Rise In Challenge is a brand, lifestyle, me...",US,tric-clothing-co,False,1,30,4000.000,Apparel,Fashion
1113925398,Inspiring girls & young women. Join me as I cr...,AU,womens-gold-medalists-rio-2016,True,1,31,37339.208,Graphic Design,Design


In [18]:
%cd /content/drive/My Drive/kickStarter
df.to_csv('cleandf.csv')

/content/drive/My Drive/kickStarter
